<a href="https://colab.research.google.com/github/DSabarish/InvestmentAnalysis/blob/main/Investment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Notes

https://medium.com/@hemansnation/11-projects-a-data-analyst-should-build-for-the-finance-domain-as-a-fresher-58bf402823cb

The project ideas mentioned in the article:
  1. Financial modeling
  2. Fraud detection
  3. Risk management
  4. Credit risk analysis
  5. Performance reporting
  6. Market analysis
  7. Customer segmentation
  8. Investment analysis
  9. Budget forecasting
  10. Product pricing analysis
  11. Compliance monitoring



8. Investment analysis
https://www.youtube.com/watch?v=sAyj_qopxHg
https://www.kaggle.com/code/ashydv/investment-opportunity-analysis-eda?source=post_page-----58bf402823cb--------------------------------


># **8. Investment analysis**





## Investment Analysis

This analysis is for a Spark Funds case study that analyzes data and derives insightful trends.

Spark Funds make investments in various companies.

The CEO of Spark Funds wants to **understand global investment trends** to make effective investment decisions.

The **objective** is to identify the **best sectors, countries, and suitable investment types**

The overall strategy is to invest where others are investing, implying that the 'best' sectors and countries are the ones 'where most investors are investing'.

Spark Funds has two minor constraints for investments:

1. It wants to invest between **5 to 15 million USD** per round of investment.
2. It wants to invest only in **English-speaking countries** because of the ease of communication with the invested companies.


#Importing Packages

In [ ]:
import pandas as pd
import numpy as np

#Loading Files as DataFrame

In [ ]:
Companies = "https://raw.githubusercontent.com/DSabarish/InvestmentAnalysis/main/companies.txt"
Rounds2 = "https://raw.githubusercontent.com/DSabarish/InvestmentAnalysis/main/rounds2.csv"

C = pd.read_csv(Companies, sep='\t', encoding='ISO-8859-1')
R = pd.read_csv(Rounds2, encoding='ISO-8859-1')

#Data Quality Exploration

In [ ]:
def DataQuality_stats(df):
    # print("FileName")
    print("Rows:", df.shape[0])
    print("Columns:", df.shape[1])
    print("Data Quality:")
    stats = pd.DataFrame({
        '#Rows': df.shape[0],
        '#Unique Values': df.nunique(),
        '#Null Values': df.isnull().sum(),
        '%Null Values': round((df.isnull().mean() * 100), 2)
    })

    return stats

In [ ]:
print("Company File")
DataQuality_stats(C)

In [ ]:
print("Round2 File")
DataQuality_stats(R)

In [ ]:
# Lowercase 'permalink' column in DataFrame C
C['permalink'] = C['permalink'].str.lower()


# Rename 'company_permalink' column to 'permalink' and the lowercasing its values in DataFrame R
R = R.rename(columns={'company_permalink': 'permalink'})
R['permalink'] = R['permalink'].str.lower()


# Perform a left join between DataFrames C and R:
# Merging C and R DataFrames using 'permalink' column as the join key,
# retaining all rows from C and including matching rows from R, with NaN values where there are no matches in R.
merged_df = pd.merge(C, R, how='left', on='permalink')

# Display the resulting DataFrame:
merged_df.head()



In [ ]:
print("Merged Dataframe")
DataQuality_stats(merged_df)

In [ ]:
# Cleaning data by removing unnecessary columns
columns_to_drop = ['funded_at', 'founded_at', 'funding_round_permalink', 'funding_round_code', 'homepage_url', 'state_code', 'region', 'city']
merged_df = merged_df.drop(columns=columns_to_drop, axis=1)

In [ ]:
# Cleaning data > Remove rows with missing values

# 'raised_amount_usd' column with 17.39% missing values
merged_df = merged_df[pd.notnull(merged_df['raised_amount_usd'])]

# 'country_code' column with 7.55% missing values
# This ensures we exclude rows without country code information, necessary for geographical analysis
merged_df = merged_df[pd.notnull(merged_df['country_code'])]

# 'category_list' column with 2.97% missing values
# This ensures we exclude rows without category list information, necessary for sector-wise analysis
merged_df = merged_df[pd.notnull(merged_df['category_list'])]

# Display the cleaned master_frame DataFrame
print("Cleaned merged_df:")
merged_df.head()


In [ ]:
print("Cleaned merged_df:")
DataQuality_stats(merged_df)

In [ ]:
round_names = R['funding_round_type'].unique()

print(round_names)

Explanation:

| Funding Round Type   | Description                                                                                        |
|----------------------|----------------------------------------------------------------------------------------------------|
| Venture              | Funding for new and growing companies with big ideas.                                              |
| Seed                 | Early funding to help start a business or develop a product.                                        |
| Undisclosed          | Funding where details are not publicly shared.                                                      |
| Equity Crowdfunding | Money raised from many people in exchange for a stake in the company.                               |
| Convertible Note     | Short-term loan that can be converted into ownership in the future.                                 |
| Private Equity       | Investment in established companies looking to grow or restructure.                                 |
| Debt Financing       | Borrowing money that needs to be repaid with interest.                                              |
| Angel                | Investment from wealthy individuals who believe in a startup's potential.                            |
| Grant                | Free money awarded for specific projects or initiatives.                                             |
| Secondary Market     | Buying and selling existing shares between investors.                                               |
| Post IPO Equity      | Funds raised by a company selling shares after going public.                                         |
| Post IPO Debt        | Debt taken on by a company after becoming publicly traded.                                           |
| Product Crowdfunding | Money raised from the public for creating a new product.                                             |
| Non-equity Assistance| Help given without expecting ownership in return, like loans or grants.                              |


In [ ]:
# prompt: export merged_df to csv

merged_df.to_csv('Cleaned_Investment_data.csv')


In [ ]:
# Grouping the master_frame DataFrame by 'funding_round_type'
funding_round_type_group = merged_df.groupby('funding_round_type')

# Calculating the average investment amount for each funding round type
type_funding = funding_round_type_group['raised_amount_usd'].mean().sort_values(ascending=False).astype(int)

# Displaying the average investment amount for each funding round type
type_funding


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set the figure size
plt.figure(figsize=(12, 8))

# Create a box plot
sns.boxplot(x='funding_round_type', y='raised_amount_usd', data=merged_df, palette='Set1')
plt.yscale('log')  # Set the y-axis scale to logarithmic for better visualization

# Add a swarm plot on top of the box plot
sns.swarmplot(x='funding_round_type', y='raised_amount_usd', data=merged_df, color='black', alpha=0.5)

# Set plot labels and title
plt.title('Distribution of Raised Amount USD by Funding Round Type')
plt.xlabel('Funding Round Type')
plt.ylabel('Raised Amount (USD)')
plt.xticks(rotation=45)  # Rotate the x-axis labels for better readability

# Show the plot
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set the style of the plot
sns.set_style("whitegrid")

# Create the box plot
plt.figure(figsize=(12, 6))
sns.boxplot(x='funding_round_type', y='raised_amount_usd', data=merged_df)
plt.yscale('log')  # Set the y-axis scale to logarithmic for better visualization
plt.title('Distribution of Raised Amount USD by Funding Round Type')
plt.xlabel('Funding Round Type')
plt.ylabel('Raised Amount (USD)')
plt.xticks(rotation=45)  # Rotate the x-axis labels for better readability
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set the style of the plot
sns.set_style("whitegrid")

# Create the box plot
plt.figure(figsize=(12, 6))
sns.boxplot(x='funding_round_type', y='raised_amount_usd', data=merged_df)
plt.yscale('log')  # Set the y-axis scale to logarithmic for better visualization
plt.title('Distribution of Raised Amount USD by Funding Round Type')
plt.xlabel('Funding Round Type')
plt.ylabel('Raised Amount (USD)')
plt.xticks(rotation=45)  # Rotate the x-axis labels for better readability
plt.show()


In [ ]:
# Create the violin plot
plt.figure(figsize=(12, 6))
sns.violinplot(x='funding_round_type', y='raised_amount_usd', data=merged_df)
plt.yscale('log')  # Set the y-axis scale to logarithmic for better visualization
plt.title('Distribution of Raised Amount USD by Funding Round Type')
plt.xlabel('Funding Round Type')
plt.ylabel('Raised Amount (USD)')
plt.xticks(rotation=45)  # Rotate the x-axis labels for better readability
plt.show()


In [ ]:
# Create the swarm plot
plt.figure(figsize=(12, 6))
sns.swarmplot(x='funding_round_type', y='raised_amount_usd', data=merged_df)
plt.yscale('log')  # Set the y-axis scale to logarithmic for better visualization
plt.title('Distribution of Raised Amount USD by Funding Round Type')
plt.xlabel('Funding Round Type')
plt.ylabel('Raised Amount (USD)')
plt.xticks(rotation=45)  # Rotate the x-axis labels for better readability
plt.show()


To explain the differences between venture capital, seed funding, and private equity in an Excel table format, you can create a table with columns for each type of funding and rows for their key characteristics. Here's how you could structure the table:

| Characteristic          | Venture Capital | Seed Funding   | Private Equity |
|-------------------------|-----------------|----------------|----------------|
| Stage of Company        | Early-stage     | Early-stage    | Mature         |
| Investment Size         | Moderate to large | Small to moderate | Large           |
| Company Development     | High-growth potential | Early development | Established     |
| Investor Requirements  | High potential for ROI | Growth potential | Profitability, cash flow |
| Equity Ownership       | Equity stake in exchange for funding | Equity stake or convertible debt | Equity stake or control |
| Typical Sector Focus   | Technology, biotech, high-growth industries | Varied | Varied, often non-tech sectors |
| Risk Level             | High            | High           | Moderate to High |
| Investment Timeline    | Medium to Long-term | Short to Medium-term | Medium to Long-term |
| Exit Strategy Options  | IPO, acquisition | Acquisition, next funding round | IPO, sale to another company, recapitalization |

This table provides a comparison of the key differences between venture capital, seed funding, and private equity across various aspects such as stage of company, investment size, investor requirements, risk level, and exit strategy options. It can help understand how each type of funding differs in terms of their target companies, investment strategies, and objectives.